# Set up packages for HW1
# Trained on Kaggle

In [1]:
!pip install --upgrade gdown tqdm

# About the Dataset

In [2]:
# Download dataset
!gdown 1Ijt2MqvY9knLkLYcgjaUwSUOClWhtX37 -O hw1_data.zip

# Unzip the downloaded zip file
# This may take some time.
!unzip -q ./hw1_data.zip

Downloading...
From (uriginal): https://drive.google.com/uc?id=1Ijt2MqvY9knLkLYcgjaUwSUOClWhtX37
From (redirected): https://drive.google.com/uc?id=1Ijt2MqvY9knLkLYcgjaUwSUOClWhtX37&confirm=t&uuid=a9fabc91-66c6-426a-9610-c7f565de9fa7
To: /kaggle/working/hw1_data.zip
100%|███████████████████████████████████████| 1.13G/1.13G [00:04<00:00, 259MB/s]


In [3]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import os
import imageio
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image #PIL包含在pillow這個函式庫
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
# This is for the progress bar.
from tqdm import tqdm
import random

In [4]:
num_classes = 7
def bilinear_kernel(in_channels, out_channels, kernel_size):
    factor = (kernel_size + 1) // 2
    if kernel_size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = (torch.arange(kernel_size).reshape(-1, 1),
          torch.arange(kernel_size).reshape(1, -1))
    filt = (1 - torch.abs(og[0] - center) / factor) * \
           (1 - torch.abs(og[1] - center) / factor)
    weight = torch.zeros((in_channels, out_channels,
                          kernel_size, kernel_size))
    weight[range(in_channels), range(out_channels), :, :] = filt
    return weight

vgg16_backbone = models.vgg16(weights=None)
W = bilinear_kernel(num_classes, num_classes, 64)

class fcn32s(nn.Module):
    def __init__(self, n_class=21):
        super(fcn32s, self).__init__()
        # vgg16
        self.features = vgg16_backbone.features # 1/32

        # fc6
        self.fc6 = nn.Conv2d(512, 4096, 7, padding=3)
        self.relu6 = nn.ReLU(inplace=True)
        self.drop6 = nn.Dropout2d()

        # fc7
        self.fc7 = nn.Conv2d(4096, 4096, 1)
        self.relu7 = nn.ReLU(inplace=True)
        self.drop7 = nn.Dropout2d()
        
        # Conv2d
        self.score_fr = nn.Conv2d(4096, n_class, 1)
        
        # ConvTranspose2d
        self.upscore = nn.ConvTranspose2d(n_class, n_class, kernel_size=64, stride=32, padding=16)
        
        # 全卷积网络用双线性插值的上采样初始化转置卷积层
        self.upscore.weight.data.copy_(W)
    

    def forward(self, x):
        
        h = self.features(x)
        
        h = self.relu6(self.fc6(h))
        h = self.drop6(h)

        h = self.relu7(self.fc7(h))
        h = self.drop7(h)

        h = self.score_fr(h)

        h = self.upscore(h)
        # h = h[:, :, 19:19 + x.size()[2], 19:19 + x.size()[3]].contiguous()

        return h

In [5]:
""" DeepLabv3 Model download and change the head for your prediction"""
from torchvision.models.segmentation.deeplabv3 import DeepLabHead, FCNHead
mv3_backbone = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.IMAGENET1K_V1, dilated=True)
mv3_backbone = mv3_backbone.features
stage_indices = [0] + [i for i, b in enumerate(mv3_backbone) if getattr(b, "_is_cn", False)] + [len(mv3_backbone) - 1]
out_pos = stage_indices[-1]  # use C5 which has output_stride = 16
out_inplanes = mv3_backbone[out_pos].out_channels

aux_pos = stage_indices[-4]  # use C2 here which has output_stride = 8
aux_inplanes = mv3_backbone[aux_pos].out_channels

def createDeepLabv3_mv3(outputchannels):
    """DeepLabv3 class with custom head
    Args:
        outputchannels (int, optional): The number of output channels
        in your dataset masks. Defaults to 1.
    Returns:
        model: Returns the DeepLabv3 model with the ResNet101 backbone.
    """
    model = models.segmentation.deeplabv3_mobilenet_v3_large(weights=models.segmentation.DeepLabV3_MobileNet_V3_Large_Weights.DEFAULT,
                                                    progress=True)

    model.aux_classifier = FCNHead(aux_inplanes, outputchannels)
    model.classifier = DeepLabHead(out_inplanes, outputchannels)

    return model


def createDeepLabv3_r50(outputchannels):
    """DeepLabv3 class with custom head
    Args:
        outputchannels (int, optional): The number of output channels
        in your dataset masks. Defaults to 1.
    Returns:
        model: Returns the DeepLabv3 model with the ResNet101 backbone.
    """
    model = models.segmentation.deeplabv3_resnet50(weights=models.segmentation.DeepLabV3_ResNet50_Weights.DEFAULT,
                                                    progress=True)

    model.classifier = DeepLabHead(2048, outputchannels)

    return model

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 201MB/s]


In [6]:
# set a random seed for reproducibility
myseed = 6666  
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

In [7]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [8]:
def read_masks(seg):
    masks = np.empty((512, 512)) # Return an array of zeros with the same shape and type as a given array
    mask = (seg >= 128).astype(int)  # 將mask中像素值大於等於128的元素轉為整數1，否則數值是原來的整數0
    mask = 4 * mask[:, :, 0] + 2 * mask[:, :, 1] + mask[:, :, 2]
    masks[mask == 3] = 0  # (Cyan: 011) Urban land 
    masks[mask == 6] = 1 # (Yellow: 110) Agriculture land 
    masks[mask == 5] = 2  # (Purple: 101) Rangeland 
    masks[mask == 2] = 3  # (Green: 010) Forest land 
    masks[mask == 1] = 4  # (Blue: 001) Water 
    masks[mask == 7] = 5  # (White: 111) Barren land 
    masks[mask == 0] = 6  # (Black: 000) Unknown
    return masks

In [9]:
# It is important to do data augmentation in training.
# However, not every augmentation is useful.
# Please think about what kind of augmentation is helpful for this recognition of this task.
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    # transforms.RandomHorizontalFlip(), #影像 RandomHorizontalFlip 和 RandomVerticalFlip
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # transforms.RandomHorizontalFlip(),
])

train_target_tfm = transforms.Compose([
    # transforms.ToPILImage(),
    #影像 RandomHorizontalFlip 和 RandomVerticalFlip
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
])

# We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_target_tfm = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

# Datasets

In [10]:
from PIL import Image
import torchvision
from torchvision import transforms as T
import random

class RandomHorizontalFlip(object):
    def __init__(self, flip_prob=0.5):
        self.flip_prob = flip_prob
 
    def __call__(self, image, target=None):
        if random.random() < self.flip_prob:
            image = torchvision.transforms.functional.hflip(image)
            if target is not None:
                target = torchvision.transforms.functional.hflip(target)
        return image, target
random_horizontal_flip = RandomHorizontalFlip(flip_prob=0.5)

In [11]:
import imageio

class hw1_3_Dataset(Dataset):
    import imageio
    def __init__(self,path,tfm=test_tfm, test=True):
        super(hw1_3_Dataset).__init__()
        self.path = path
        self.files = sorted([x.split(".")[0].split('_')[0] for x in os.listdir(path) if x.endswith(".jpg")])
        self.imgs = [os.path.join(path,(x)+"_sat.jpg") for x in self.files]
        self.masks = [os.path.join(path,(x)+"_mask.png") for x in self.files]
        
        self.transform = tfm
        self.test = test
    def __len__(self):
        return len(self.files)

    def __getitem__(self,idx):
        img_name = self.imgs[idx] # fname = 31_327
        mask_name = self.masks[idx]
        im = Image.open(img_name)
        im = self.transform(im)
        if not self.test:
            try:
                mask = imageio.imread(mask_name)
                mask = read_masks(mask)
                mask = torch.from_numpy(mask).to(dtype=torch.long)
                im, mask = random_horizontal_flip(im, mask)
            except:
                mask = im
        else:
            mask = imageio.imread(mask_name)
            mask = read_masks(mask)
            mask = torch.from_numpy(mask).to(dtype=torch.long)
        
        return im,mask

# Dataloader(Mini-ImageNet)

In [12]:
# Construct train and valid datasets.
# The argument "loader" tells how torchvision reads the data.
# The number of batch size.
batch_size = 16
train_set = hw1_3_Dataset(path="./hw1_data/p3_data/train", tfm=train_tfm, test=False)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

val_set = hw1_3_Dataset(path="./hw1_data/p3_data/validation", tfm=test_tfm, test=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

In [13]:
num_classes = 7
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
vgg16_fcn32s = fcn32s(n_class=num_classes)
# model = vgg16_fcn32s
model = createDeepLabv3_mv3(outputchannels=num_classes)
# model = createDeepLabv3_r50(outputchannels=num_classes)
model.to(device)

# The number of training epochs.
n_epochs = 80

# weight decay
wd_num = 1e-5

# MixUp alpha(α \alphaα在0.2 ~ 2之間效果都差不多，表示mixup對α \alphaα參數並不是很敏感。但如果α \alphaα過小，等於沒有進行mixup的原始數據，如果α \alphaα過大，等於所有輸入都是各取一半混合)
alpha = 0

# If no improvement in 'patience' epochs, early stop.
patience = 15

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=wd_num)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=wd_num)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.1, patience=10)

ues_lrscheduling = True
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=10)


ckpt_name = "exp8_createDeepLabv3_mv3_best.ckpt"
first_ckpt_name = 'exp8_createDeepLabv3_mv3_first.ckpt '
middle_ckpt_name = 'exp8_createDeepLabv3_mv3_middle.ckpt '
final_ckpt_name = 'exp8_createDeepLabv3_mv3_final.ckpt '

Downloading: "https://download.pytorch.org/models/deeplabv3_mobilenet_v3_large-fc3c493d.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_mobilenet_v3_large-fc3c493d.pth
100%|██████████| 42.3M/42.3M [00:00<00:00, 163MB/s]


# Create Model and Configurations

In [14]:
import numpy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


class BinaryFocalLoss(nn.Module):
    """
    This is a implementation of Focal Loss with smooth label cross entropy supported which is proposed in
    'Focal Loss for Dense Object Detection. (https://arxiv.org/abs/1708.02002)'
        Focal_Loss= -1*alpha*(1-pt)*log(pt)
    :param alpha: (tensor) 3D or 4D the scalar factor for this criterion
    :param gamma: (float,double) gamma > 0 reduces the relative loss for well-classified examples (p>0.5) putting more
                    focus on hard misclassified example
    :param reduction: `none`|`mean`|`sum`
    :param **kwargs
        balance_index: (int) balance class index, should be specific when alpha is float
    """

    def __init__(self, alpha=0.25, gamma=2, ignore_index=None, reduction='mean', **kwargs):
        super(BinaryFocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.smooth = 1e-6  # set '1e-4' when train with FP16
        self.ignore_index = ignore_index
        self.reduction = reduction

        assert self.reduction in ['none', 'mean', 'sum']

        # if self.alpha is None:
        #     self.alpha = torch.ones(2)
        # elif isinstance(self.alpha, (list, np.ndarray)):
        #     self.alpha = np.asarray(self.alpha)
        #     self.alpha = np.reshape(self.alpha, (2))
        #     assert self.alpha.shape[0] == 2, \
        #         'the `alpha` shape is not match the number of class'
        # elif isinstance(self.alpha, (float, int)):
        #     self.alpha = np.asarray([self.alpha, 1.0 - self.alpha], dtype=np.float).view(2)

        # else:
        #     raise TypeError('{} not supported'.format(type(self.alpha)))

    def forward(self, output, target):
        prob = torch.sigmoid(output)
        prob = torch.clamp(prob, self.smooth, 1.0 - self.smooth)

        valid_mask = None
        if self.ignore_index is not None:
            valid_mask = (target != self.ignore_index).float()

        pos_mask = (target == 1).float()
        neg_mask = (target == 0).float()
        if valid_mask is not None:
            pos_mask = pos_mask * valid_mask
            neg_mask = neg_mask * valid_mask

        pos_weight = (pos_mask * torch.pow(1 - prob, self.gamma)).detach()
        pos_loss = -pos_weight * torch.log(prob)  # / (torch.sum(pos_weight) + 1e-4)

        neg_weight = (neg_mask * torch.pow(prob, self.gamma)).detach()
        neg_loss = -self.alpha * neg_weight * F.logsigmoid(-output)  # / (torch.sum(neg_weight) + 1e-4)
        loss = pos_loss + neg_loss
        loss = loss.mean()
        return loss


class FocalLoss_Ori(nn.Module):
    """
    This is a implementation of Focal Loss with smooth label cross entropy supported which is proposed in
    'Focal Loss for Dense Object Detection. (https://arxiv.org/abs/1708.02002)'
    Focal_Loss= -1*alpha*((1-pt)**gamma)*log(pt)
    Args:
        num_class: number of classes
        alpha: class balance factor
        gamma:
        ignore_index:
        reduction:
    """

    def __init__(self, num_class, alpha=None, gamma=2, ignore_index=None, reduction='mean'):
        super(FocalLoss_Ori, self).__init__()
        self.num_class = num_class
        self.gamma = gamma
        self.reduction = reduction
        self.smooth = 1e-4
        self.ignore_index = ignore_index
        self.alpha = alpha
        if alpha is None:
            self.alpha = torch.ones(num_class, )
        elif isinstance(alpha, (int, float)):
            self.alpha = torch.as_tensor([alpha] * num_class)
        elif isinstance(alpha, (list, np.ndarray)):
            self.alpha = torch.as_tensor(alpha)
        if self.alpha.shape[0] != num_class:
            raise RuntimeError('the length not equal to number of class')

        # if isinstance(self.alpha, (list, tuple, np.ndarray)):
        #     assert len(self.alpha) == self.num_class
        #     self.alpha = torch.Tensor(list(self.alpha))
        # elif isinstance(self.alpha, (float, int)):
        #     assert 0 < self.alpha < 1.0, 'alpha should be in `(0,1)`)'
        #     assert balance_index > -1
        #     alpha = torch.ones((self.num_class))
        #     alpha *= 1 - self.alpha
        #     alpha[balance_index] = self.alpha
        #     self.alpha = alpha
        # elif isinstance(self.alpha, torch.Tensor):
        #     self.alpha = self.alpha
        # else:
        #     raise TypeError('Not support alpha type, expect `int|float|list|tuple|torch.Tensor`')

    def forward(self, logit, target):
        # assert isinstance(self.alpha,torch.Tensor)\
        N, C = logit.shape[:2]
        alpha = self.alpha.to(logit.device)
        prob = F.softmax(logit, dim=1)
        if prob.dim() > 2:
            # N,C,d1,d2 -> N,C,m (m=d1*d2*...)
            prob = prob.view(N, C, -1)
            prob = prob.transpose(1, 2).contiguous()  # [N,C,d1*d2..] -> [N,d1*d2..,C]
            prob = prob.view(-1, prob.size(-1))  # [N,d1*d2..,C]-> [N*d1*d2..,C]
        ori_shp = target.shape
        target = target.view(-1, 1)  # [N,d1,d2,...]->[N*d1*d2*...,1]
        valid_mask = None
        if self.ignore_index is not None:
            valid_mask = target != self.ignore_index
            target = target * valid_mask

        # ----------memory saving way--------
        prob = prob.gather(1, target).view(-1) + self.smooth  # avoid nan
        logpt = torch.log(prob)
        # alpha_class = alpha.gather(0, target.view(-1))
        alpha_class = alpha[target.squeeze().long()]
        class_weight = -alpha_class * torch.pow(torch.sub(1.0, prob), self.gamma)
        loss = class_weight * logpt
        if valid_mask is not None:
            loss = loss * valid_mask.squeeze()

        if self.reduction == 'mean':
            loss = loss.mean()
            if valid_mask is not None:
                loss = loss.sum() / valid_mask.sum()
        elif self.reduction == 'none':
            loss = loss.view(ori_shp)
        return loss
    
FL_criterion = FocalLoss_Ori(num_class=7, alpha=0.25)

In [ ]:
## Initialize trackers, these are not parameters and should not be changed
stale = 0
best_mIoU = 0

for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    tp_fp = np.zeros(6)
    tp_fn = np.zeros(6)
    tp = np.zeros(6)
    mean_iou = 0
    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()
        imgs, labels = imgs.to(device), labels.to(device)
        # imgs, targets_a, targets_b, lam = mixup_data(imgs, labels, alpha)
        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs)['out']
        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        # loss = mixup_criterion(criterion, logits, targets_a, targets_b, lam)
        loss = criterion(logits, labels)

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()
    
        # Compute the accuracy for current batch.
        # acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
        label_pred = logits.argmax(dim=1).detach().cpu().numpy()
        
        label_true = labels.detach().cpu().numpy()
        # Record the loss and accuracy.
        train_loss.append(loss.item())    

        for i in range(6):
            tp_fp_num = np.sum(label_pred == i)
            tp_fn_num = np.sum(label_true == i)
            tp_num = np.sum((label_pred == i) * (label_true == i))
            tp_fp[i] += tp_fp_num
            tp_fn[i] += tp_fn_num
            tp[i] += tp_num
    print(tp_fp)      
    for i in range(6):
        iou = tp[i] / (tp_fp[i] + tp_fn[i] - tp[i])
        mean_iou += iou / 6
        print('class #%d : %1.5f'%(i, iou))
    print('\ntrain_mean_iou: %f\n' % mean_iou)
    
    train_loss = sum(train_loss) / len(train_loss)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, mIoU = {mean_iou:.5f}")
    
    if ues_lrscheduling:
        scheduler.step(train_loss)
        
    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    tp_fp = np.zeros(6)
    tp_fn = np.zeros(6)
    tp = np.zeros(6)
    mean_iou = 0

    # Iterate the validation set by batches.
    for batch in tqdm(val_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device))['out']
        
        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        label_pred = logits.argmax(dim=1).detach().cpu().numpy()
        label_true = labels.detach().cpu().numpy()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        for i in range(6):
            tp_fp_num = np.sum(label_pred == i)
            tp_fn_num = np.sum(label_true == i)
            tp_num = np.sum((label_pred == i) * (label_true == i))
            tp_fp[i] += tp_fp_num
            tp_fn[i] += tp_fn_num
            tp[i] += tp_num
        #break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    
    print(tp_fp)      
    for i in range(6):
        iou = tp[i] / (tp_fp[i] + tp_fn[i] - tp[i])
        mean_iou += iou / 6
        print('class #%d : %1.5f'%(i, iou))
    print('\nvalidation_mean_iou: %f\n' % mean_iou)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, mIoU = {mean_iou:.5f}")
        
    # update logs
    if mean_iou > best_mIoU:
        with open("./sample_best_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, mIoU = {mean_iou:.5f} -> best")
    else:
        with open("./sample_best_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, mIoU = {mean_iou:.5f}")

    # save models
    if mean_iou > best_mIoU:
        best_epoch = epoch+1
        print(f"Best model found at epoch {best_epoch}, saving model")
        torch.save(model.state_dict(), ckpt_name) # only save best to prevent output memory exceed error
        best_mIoU = mean_iou
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping at no.{epoch+1} epoch, best epoch at {best_epoch}.")
            break
            
    if (epoch+1) == 1:
        print(f"First model found at epoch {epoch+1}, saving model")
        torch.save(model.state_dict(), first_ckpt_name)
    if (epoch+1) == (2):
        print(f"Middle model found at epoch {epoch+1}, saving model")
        torch.save(model.state_dict(), middle_ckpt_name)
    if (epoch+1) == n_epochs:
        print(f"Final model found at epoch {epoch+1}, saving model")
        torch.save(model.state_dict(), final_ckpt_name)

# Visualization

In [ ]:
num_classes = 7
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

model = createDeepLabv3_mv3(outputchannels=num_classes)
model.to(device)
model.eval()
model.load_state_dict(torch.load('/kaggle/input/exp2-createdeeplabv3-mv3-best-ckpt/exp2_createDeepLabv3_mv3_best.ckpt'))

In [ ]:
from PIL import Image
from torchvision import transforms
filename = ['./hw1_data/p3_data/validation/0013_sat.jpg','./hw1_data/p3_data/validation/0062_sat.jpg','./hw1_data/p3_data/validation/0104_sat.jpg']
filename = filename[1]
input_image = Image.open(filename)
input_image = input_image.convert("RGB")
input_tensor = test_tfm(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

input_batch = input_batch.to(device)

with torch.no_grad():
    output = model(input_batch)['out'][0]
output_predictions = output.argmax(0).detach().cpu().numpy()
viz_masks = np.repeat(output_predictions[:, :, np.newaxis], 3, axis=2)

In [ ]:
import os
import argparse
import scipy.ndimage
import imageio.v2 as imageio

import numpy as np 

from matplotlib import colors as mcolors

voc_cls = {'urban': 0, 
           'rangeland': 2,
           'forest': 3,  
           'unknown': 6,  
           'barreb land': 5,  
           'Agriculture land': 1,  
           'water': 4,
           } 
cls_color = {
    0:  [0, 255, 255],
    1:  [255, 255, 0],
    2:  [255, 0, 255],
    3:  [0, 255, 0],
    4:  [0, 0, 255],
    5:  [255, 255, 255],
    6: [0, 0, 0],
}
def mask_edge_detection(mask, edge_width):
    h = mask.shape[0]
    w = mask.shape[1]

    edge_mask = np.zeros((h,w))
    
    for i in range(h):
        for j in range(1,w):
            j_prev = j - 1 
            # horizontal #
            if not mask[i][j] == mask[i][j_prev]: # horizontal
                if mask[i][j]==1: # 0 -> 1
                    edge_mask[i][j] = 1
                    for add in range(1,edge_width):
                        if j + add < w and mask[i][j+add] == 1:
                            edge_mask[i][j+add] = 1
                        
                else : # 1 -> 0
                    edge_mask[i][j_prev] = 1
                    for minus in range(1,edge_width):
                        if j_prev - minus >= 0 and mask[i][j_prev - minus] == 1: 
                            edge_mask[i][j_prev - minus] = 1
            # vertical #
            if not i == 0 :
                i_prev = i - 1
                if not mask[i][j] == mask[i_prev][j]: 
                    if mask[i][j]==1: # 0 -> 1
                        edge_mask[i][j] = 1 
                        for add in range(1,edge_width):
                            if i + add < h and mask[i+add][j] == 1:
                                edge_mask[i+add][j] = 1 
                    else : # 1 -> 0
                        edge_mask[i_prev][j] = 1
                        for minus in range(1,edge_width):
                            if i_prev - minus >= 0 and mask[i_prev-minus][j] == 1:
                                edge_mask[i_prev-minus][j] == 1
    return edge_mask

def viz_data(im, seg, color, inner_alpha = 0.3, edge_alpha = 1, edge_width = 5):
     
    edge = mask_edge_detection(seg, edge_width)

    color_mask = np.zeros((edge.shape[0]*edge.shape[1], 3))
    l_loc = np.where(seg.flatten() == 1)[0]
    color_mask[l_loc, : ] = color
    color_mask = np.reshape(color_mask, im.shape)
    mask = np.concatenate((seg[:,:,np.newaxis],seg[:,:,np.newaxis],seg[:,:,np.newaxis]), axis = -1)
    
    color_edge = np.zeros((edge.shape[0]*edge.shape[1], 3))
    l_col = np.where(edge.flatten() == 1)[0]
    color_edge[l_col,:] = color
    color_edge = np.reshape(color_edge, im.shape)
    edge = np.concatenate((edge[:,:,np.newaxis],edge[:,:,np.newaxis],edge[:,:,np.newaxis]), axis = -1)


    im_new = im*(1-mask) + im*mask*(1-inner_alpha) + color_mask * inner_alpha
    im_new =  im_new*(1-edge) + im_new*edge*(1-edge_alpha) + color_edge*edge_alpha

    return im_new 

def arg_parse():
    parser = argparse.ArgumentParser(description='Tools to visualize semantic segmentation map.')

    # Datasets parameters
    parser.add_argument('--sat_path', type=str, default='', 
                    help="path to RGB satellite image")
    parser.add_argument('--mask_path', type=str, default='', 
                    help="path to mask")

    args = parser.parse_args()

    return args

def read_masks(seg):
    masks = np.zeros_like(seg) # Return an array of zeros with the same shape and type as a given array
    mask = (seg >= 128).astype(int)  # 將mask中像素值大於等於128的元素轉為整數1，否則數值是原來的整數0
    mask = 4 * mask[:, :, 0] + 2 * mask[:, :, 1] + mask[:, :, 2]
    masks[mask == 3] = 0  # (Cyan: 011) Urban land 
    masks[mask == 6] = 1  # (Yellow: 110) Agriculture land 
    masks[mask == 5] = 2  # (Purple: 101) Rangeland 
    masks[mask == 2] = 3  # (Green: 010) Forest land 
    masks[mask == 1] = 4  # (Blue: 001) Water 
    masks[mask == 7] = 5  # (White: 111) Barren land 
    masks[mask == 0] = 6  # (Black: 000) Unknown
    return masks



if __name__ == '__main__':
    cmap = cls_color
    img = imageio.imread(filename)
    masks = viz_masks
    cs = np.unique(masks)

    for c in cs:
        mask = np.zeros((img.shape[0], img.shape[1]))
        ind = np.where(masks == c)
        mask[ind[0], ind[1]] = 1
        img = viz_data(img, mask, color=cmap[c])
        imageio.imsave('./exp.png', np.uint8(img))
